### Recommendation System

1. Data handling and preparation, mention which service tools, algo you will use

I would use those services and tools, if possible, that are already in the system. So that the expansion of functionality would have a minimal impact on the stability and cost of the entire system. For example, if it is possible to pull the ml model directly from the php engine, we will do this, and not raise a separate service for this. If the data is in RDS, and Amazon SageMaker is not expensive, we will consider the model right there. Or if we already have a Spark cluster, we can count on it.
I have practical experience with RDS + Airflow + FastAPI service, but all this was already in the system, I just redistributed the resources. If not, then this is not the best combination.

2. Describe algorithm you will implement to construct the different product suggestion,
which machine learning algorithm you will user

Since it is more convenient for me to think about specific data and with the help of code - I generated toy data and showed how I would explore them. Due to the lack of time and the simplicity of the prototype, some stages of the pipeline are missing (data cleaning, tuning, comparing metrics).
However, this pipeline reflects the very approach of finding the best solution and it is like this: we move from the simplest algorithm to the most complex one or a combination of them and compare the metrics.
According to this logic, after the Content-based and Matrix factorization algorithms, I should have used a hybrid model (which can process both the rating matrix and the vector of product or user characteristics together). An example of such a model would be Lightfm. Thus, only by completing the study of real data and having tried several basic algorithms, you can answer the question of which is better. For a particular sample data, it is obvious that a hybrid lightfm model would be the best solution, due to the use of all the information.

3. Describe the data models you will be creating, where to store them, when to refresh
them / how

Depends entirely on the answers to the question from point 1. The frequency of data updates depends on the requirements of the customer

# 0. Generate data

In [1]:
import pandas as pd
import matplotlib.colors as mcolors
import numpy as np
import random
import string


from datetime import datetime


In [2]:
cloth = ["jeans", "sweater", "shoes", "shirt", "shorts", "jacket", "cloak", "sandals", "cap", "jacket"]
colors = list(mcolors.CSS4_COLORS.keys())

def get_rand_feature(cloth, colors):    
    res = {}
    for c in range(2):
        color_num = np.random.randint(0, len(colors))
        res[colors[color_num]] = round(np.random.rand(), 1)
    cloth_num = np.random.randint(0, len(cloth))
    res[cloth[cloth_num]] = round(np.random.rand(), 1)
    return res

In [3]:
def get_rand_transaction(df_products, users, max_quantity):
    random.seed(datetime.now().timestamp())
    product_num = np.random.randint(0, len(df_products))
    user_num = np.random.randint(0, len(users))
    user = users[user_num]
    quantity = np.random.randint(1, max_quantity)
    return [df_products.loc[product_num, "store_id"],
            str(df_products.loc[product_num, "product"]) + "-" + str(random.randint(0, 100)),
            user,
            df_products.loc[product_num, "product"],
            quantity
           ]



In [4]:
N_products = 1000
N_users = 10000
N_transaction = 100000
max_quantity = 10
products = ["id_" + ''.join(random.choices(string.ascii_lowercase, k=2)) for x in range(N_products)]
users = ["ud_" + str(random.randint(100, 999)) for x in range(N_users)]
stores = np.random.randint(1, 5, N_products)

In [5]:
Products = pd.DataFrame(columns=["product", "store_id"],
                        data=list(zip(products, stores)))

Product_Features = Products.copy(deep=True)
Product_Features["feature"] = [get_rand_feature(cloth, colors) 
                               for x in range(len(Product_Features))]

Transaction_History = pd.DataFrame(columns=["store_id", "transaction_id", "user_id", "product", "quantity"],
                                   data=[get_rand_transaction(Products, users, max_quantity) 
                                         for x in range(N_transaction)])

Users_Features = pd.DataFrame(columns=["user_id", "json_product_features"])
Users_Features = pd.merge(Transaction_History, Product_Features, on="product")[["user_id", "feature"]]
Users_Features = Users_Features.groupby("user_id", as_index=False).agg({'feature': list})

# 1. Content-based recommendation system

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
df = pd.DataFrame(columns=cloth + colors, index=Product_Features["product"].unique())
df.fillna(0, inplace=True)

for ind in Product_Features.index:
    product = Product_Features.loc[ind, "product"]
    feature = Product_Features.loc[ind, "feature"]
    for k, v in feature.items():
        df.loc[product, k] = df.loc[product, k] + v
print(df.shape)
df.head(3)        

(521, 158)


,jeans,sweater,shoes,shirt,shorts,jacket,cloak,sandals,cap,jacket,...,teal,thistle,tomato,turquoise,violet,wheat,white,whitesmoke,yellow,yellowgreen
id_rf,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.5,0.0,0.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
id_oa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
id_xo,0.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0,0.0,0.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
cosine_sim = cosine_similarity(df, df)
cosine_sim

array([[1.        , 0.        , 0.46355202, ..., 0.        , 0.        ,
        0.34890892],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.46355202, 0.        , 1.        , ..., 0.        , 0.        ,
        0.43992582],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.34890892, 0.        , 0.43992582, ..., 0.        , 0.        ,
        1.        ]])

In [10]:
def similar(name, df, cosine_sim, N, log="PRODUCT"):
    ind_2_product = dict(enumerate(df.index))
    product_2_ind = {v:k for (k,v) in ind_2_product.items()}
    ind = product_2_ind[name]
    best_res = sorted(list(enumerate(cosine_sim[ind])), key=lambda x: x[1], reverse=True)[1:1+N]
    best_inds = [p[0] for p in best_res]
    best_names = [ind_2_product[x] for x in best_inds]
    if (log == 'PRODUCT') or (log == 'USER'):
        temp = df.loc[name, :]
        vec = temp[temp !=0]
        print(f"CURRENT {log}: \n name={name}, ind={ind}, \n")
        print(f"vec=\n{vec.to_string()} \n")
        print("SIMILAR {log}:")
        counter = 0
        for (i,cos) in best_res:
            counter+=1
            temp = df.loc[ind_2_product[i], :]
            vec = temp[temp !=0]
            print(f"TOP {counter}: name={ind_2_product[i]}, ind={i}, cos={cos},\n vec=\n{vec.to_string()}")
    return best_names

In [11]:
similar("id_dh", df, cosine_sim, 10, log="PRODUCT")

CURRENT PRODUCT: 
 name=id_dh, ind=419, 

vec=
sweater       0.1
shorts        0.4
sandals       0.4
cyan          0.2
darksalmon    0.6
gainsboro     0.6
green         0.2
peru          0.8 

SIMILAR {log}:
TOP 1: name=id_ti, ind=460, cos=0.5314940034527338,
 vec=
shorts       0.2
gainsboro    0.2
TOP 2: name=id_rc, ind=217, cos=0.5094468485704916,
 vec=
sandals       0.8
bisque        0.4
darksalmon    0.9
TOP 3: name=id_gu, ind=184, cos=0.44664692275015383,
 vec=
shorts               0.4
mediumspringgreen    0.3
peru                 0.2
TOP 4: name=id_hv, ind=249, cos=0.42388891641011267,
 vec=
shirt    0.1
peru     0.9
pink     0.9
TOP 5: name=id_fc, ind=66, cos=0.4075557568177074,
 vec=
shorts        1.2
sandals       0.9
brown         0.3
burlywood     0.1
lightcoral    0.1
lightgreen    0.2
TOP 6: name=id_ci, ind=160, cos=0.3757950707310682,
 vec=
jeans            0.7
shorts           0.7
gainsboro        0.9
ghostwhite       0.5
orchid           0.8
palegoldenrod    0.1
TOP 7: 

['id_ti',
 'id_rc',
 'id_gu',
 'id_hv',
 'id_fc',
 'id_ci',
 'id_hl',
 'id_se',
 'id_ys',
 'id_hy']

In [12]:
Users_Features

,user_id,feature
0,ud_100,"[{'saddlebrown': 0.3, 'mistyrose': 0.5, 'short..."
1,ud_101,"[{'yellow': 0.8, 'limegreen': 0.3, 'sweater': ..."
2,ud_102,"[{'yellow': 0.8, 'limegreen': 0.3, 'sweater': ..."
3,ud_103,"[{'sandybrown': 1.0, 'gold': 0.2, 'sandals': 0..."
4,ud_104,"[{'lightcyan': 0.4, 'violet': 0.8, 'shoes': 0...."
...,...,...
895,ud_995,"[{'cadetblue': 0.2, 'aqua': 0.1, 'sweater': 0...."
896,ud_996,"[{'lightgoldenrodyellow': 0.0, 'antiquewhite':..."
897,ud_997,"[{'cadetblue': 0.2, 'aqua': 0.1, 'sweater': 0...."
898,ud_998,"[{'cadetblue': 0.2, 'aqua': 0.1, 'sweater': 0...."


In [13]:
dfu = pd.DataFrame(columns=cloth + colors, index=Users_Features["user_id"])
dfu.fillna(0, inplace=True)

for ind in Users_Features.index:
    user_id = Users_Features.loc[ind, "user_id"]
    features = Users_Features.loc[ind, "feature"]
    for i in features:
        for k, v in i.items():
            dfu.loc[user_id, k] = dfu.loc[user_id, k] + v
print(dfu.shape)
dfu.head(3)     

(900, 158)


,jeans,sweater,shoes,shirt,shorts,jacket,cloak,sandals,cap,jacket,...,teal,thistle,tomato,turquoise,violet,wheat,white,whitesmoke,yellow,yellowgreen
user_id,,,,,,,,,,,,,,,,,,,,,
ud_100,6.2,4.1,6.6,6.5,7.6,16.4,6.5,5.0,5.6,16.4,...,0.3,0.5,1.0,0.9,0.0,1.8,2.9,0.8,1.9,0.9
ud_101,16.4,8.5,16.8,8.9,16.7,28.1,16.0,21.9,10.8,28.1,...,1.6,4.1,2.7,2.5,2.2,2.9,3.1,4.3,4.7,3.0
ud_102,16.3,9.4,21.2,19.3,13.9,35.2,18.7,16.8,16.2,35.2,...,1.9,2.4,2.2,0.6,2.5,6.3,5.5,1.0,5.0,1.2


In [14]:
cosine_simu = cosine_similarity(dfu, dfu)
cosine_simu

array([[1.        , 0.91161303, 0.93530785, ..., 0.91316215, 0.92688428,
        0.91930229],
       [0.91161303, 1.        , 0.93776615, ..., 0.91056193, 0.92168688,
        0.95475326],
       [0.93530785, 0.93776615, 1.        , ..., 0.93011479, 0.93200678,
        0.9462088 ],
       ...,
       [0.91316215, 0.91056193, 0.93011479, ..., 1.        , 0.91182802,
        0.9337089 ],
       [0.92688428, 0.92168688, 0.93200678, ..., 0.91182802, 1.        ,
        0.9471804 ],
       [0.91930229, 0.95475326, 0.9462088 , ..., 0.9337089 , 0.9471804 ,
        1.        ]])

In [15]:
similar("ud_102", dfu, cosine_simu, 10, log="USER")

CURRENT USER: 
 name=ud_102, ind=2, 

vec=
jeans                   16.3
sweater                  9.4
shoes                   21.2
shirt                   19.3
shorts                  13.9
jacket                  35.2
cloak                   18.7
sandals                 16.8
cap                     16.2
jacket                  35.2
antiquewhite             0.9
aqua                     2.8
aquamarine               0.4
azure                    4.2
beige                    3.9
bisque                   3.0
black                    2.1
blanchedalmond           4.5
blue                     2.3
blueviolet               1.8
brown                    2.3
burlywood                4.0
cadetblue                0.9
chartreuse               3.9
chocolate                4.5
coral                    3.9
cornflowerblue           2.7
cornsilk                 1.5
crimson                  3.7
cyan                     2.1
darkblue                 0.9
darkcyan                 2.1
darkgoldenrod            5.5


['ud_628',
 'ud_342',
 'ud_582',
 'ud_393',
 'ud_922',
 'ud_837',
 'ud_986',
 'ud_269',
 'ud_410',
 'ud_351']

# 2. Matrix Faktorization ALS / Spark + Pandas

In [16]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

spark = SparkSession.builder.appName('RS').getOrCreate()

23/06/30 11:23:42 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.1.37 instead (on interface wlp0s20f3)
23/06/30 11:23:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/30 11:23:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/mnemonic/anaconda3/envs/test/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [17]:
th = Transaction_History.groupby(["user_id", "product"], as_index=False).agg({"quantity": sum})
sth=spark.createDataFrame(th)
indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") 
           for column in list(set(sth.columns)-set(['quantity']))]
pipeline = Pipeline(stages=indexer)
sth = pipeline.fit(sth).transform(sth)
sth.show()

+-------+-------+--------+-------------+-------------+
|user_id|product|quantity|user_id_index|product_index|
+-------+-------+--------+-------------+-------------+
| ud_100|  id_ac|       4|        866.0|        361.0|
| ud_100|  id_am|      14|        866.0|        132.0|
| ud_100|  id_ao|       3|        866.0|        239.0|
| ud_100|  id_aq|       2|        866.0|        123.0|
| ud_100|  id_ck|       4|        866.0|         65.0|
| ud_100|  id_dn|       2|        866.0|        118.0|
| ud_100|  id_eg|       6|        866.0|        513.0|
| ud_100|  id_ei|       1|        866.0|        110.0|
| ud_100|  id_ep|       5|        866.0|         50.0|
| ud_100|  id_fr|       5|        866.0|         14.0|
| ud_100|  id_gd|       8|        866.0|         40.0|
| ud_100|  id_gp|       5|        866.0|        257.0|
| ud_100|  id_gu|       6|        866.0|        447.0|
| ud_100|  id_gx|      10|        866.0|        133.0|
| ud_100|  id_hl|       4|        866.0|         61.0|
| ud_100| 

In [18]:
(train, test)=sth.randomSplit([0.8, 0.2])

In [19]:
print((train.count(), len(train.columns)))
print((test.count(), len(test.columns)))

(69156, 5)
(17437, 5)


In [20]:
als=ALS(maxIter=10, regParam=0.09, rank=50,
        userCol="user_id_index", itemCol="product_index", ratingCol="quantity",
        coldStartStrategy="drop", nonnegative=True)
model=als.fit(train)

In [21]:
evaluator=RegressionEvaluator(metricName="rmse",labelCol="quantity",predictionCol="prediction")
predictions=model.transform(test)
rmse=evaluator.evaluate(predictions)
print("RMSE="+str(rmse))

RMSE=3.9637513965987896


In [22]:
sth = sth.toPandas()
prod_2_ind = dict(sth[['product', "product_index"]].values)
ind_2_prod = {v:k for (k,v) in prod_2_ind.items()}
user_2_ind = dict(sth[['user_id', "user_id_index"]].values)
ind_2_user = {v:k for (k,v) in user_2_ind.items()}

In [23]:
rdf=model.recommendForAllUsers(10).toPandas()
top_10 = rdf.recommendations.apply(pd.Series).merge(rdf, right_index = True, left_index = True).drop(["recommendations"], axis = 1)
top_10r = top_10.melt(id_vars = ['user_id_index'], value_name = "recommendation").drop("variable", axis = 1)
top_10_unpuck = pd.concat([top_10r['recommendation'].apply(pd.Series), top_10r['user_id_index']], axis = 1)
top_10_unpuck.rename(columns={0:"product_index", 1:"rait"}, inplace=True)
top_10_unpuck["user_id"] = top_10_unpuck.user_id_index.map(ind_2_user)
top_10_unpuck["product"] = top_10_unpuck.product_index.map(ind_2_prod)

top_10_unpuck.sort_values(['user_id',"rait"], ascending=[True, False], inplace=True)
top_10_unpuck['recommendations'] = list(zip(top_10_unpuck["product"].values, top_10_unpuck["rait"].values))
res = top_10_unpuck[['user_id', 'recommendations']].groupby("user_id").agg({"recommendations": list})
res

,recommendations
user_id,
ud_100,"[(id_zx, 12.376265525817871), (id_sd, 10.90267..."
ud_101,"[(id_yo, 11.788537979125977), (id_pp, 11.66227..."
ud_102,"[(id_rd, 14.499395370483398), (id_zx, 14.18957..."
ud_103,"[(id_ly, 13.864228248596191), (id_oh, 12.47688..."
ud_104,"[(id_eu, 20.018892288208008), (id_zh, 13.22652..."
...,...
ud_995,"[(id_nm, 11.801484107971191), (id_ng, 11.26660..."
ud_996,"[(id_wr, 10.641517639160156), (id_gd, 10.44754..."
ud_997,"[(id_cq, 13.016162872314453), (id_yj, 12.01070..."


## 1. Find Similar users to a given user


#### In 1. Content-based recommendation system run 
### similar("ud_102", dfu, cosine_simu, 10, log="USER")
#### or it can be calculate in ALS or LightFM

## 2. Find Similar products to a given input product

#### In 1. Content-based recommendation system run 
### similar("id_dh", df, cosine_sim, 10, log="PRODUCT")
#### or it can be calculate in ALS or LightFM

## 3. Find product that are sold together

All sales are grouped by transactions.
further, we divide the tuples of goods into all kinds of bigrams (product, product), take the weight as the sum of the transaction divided by the number of tuples. then we sum up the same tuples, sort and get the most frequently purchased pairs (product, product)